Note: Do not run this code. It will timeout or run for 12+ hours. The messages seen down below are simply warning messages that do not impact the final result.

Necessary imports:

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time

The URL's we needed to access for player data on Baseball Reference are indexed by character. We iterate through the alphabet to get all player data. For each letter we gather all player links and store them in a dataframe.

In [ ]:
playerLinks = []

for letter in 'abcdefghijklmnopqrstuvwxyz':
  response = requests.get(f"https://www.baseball-reference.com/players/{letter}/") # get content from the page with a last names
  time.sleep(3.1)
  soup = BeautifulSoup(response.content, "html.parser")

  contentBlock = soup.find("div",
                    attrs={
                        "class": "section_content",
                        "id": "div_players_"
                        })
  allPlayers = contentBlock.find_all('p') # this is where the player information is contained

  for player in allPlayers:
    dates = re.search(r'\(\d{4}-(\d{4})\)', str(player)) # this regular expression extracts the last year a player played
    endDate = int(dates.group(1))
    if endDate < 2023 and endDate >= 1940: # filter down the list
      playerLinks.append(player.find("a")['href']) # get the link to the player webpage

with open('playerLinks', 'wb') as pickle_file:
  pickle.dump(playerLinks, pickle_file)

playerLinks

['/players/a/aardsda01.shtml',
 '/players/a/aasedo01.shtml',
 '/players/a/abadan01.shtml',
 '/players/a/abbotgl01.shtml',
 '/players/a/abbotje01.shtml',
 '/players/a/abbotji01.shtml',
 '/players/a/abbotku01.shtml',
 '/players/a/abbotky01.shtml',
 '/players/a/abbotpa01.shtml',
 '/players/a/abercre01.shtml',
 '/players/a/abernbr01.shtml',
 '/players/a/abnersh01.shtml',
 '/players/a/abregjo01.shtml',
 '/players/a/abreubo01.shtml',
 '/players/a/abreuju01.shtml',
 '/players/a/abreuto01.shtml',
 '/players/a/abreuwi01.shtml',
 '/players/a/accarje01.shtml',
 '/players/a/acevejo01.shtml',
 '/players/a/aceveju01.shtml',
 '/players/a/aceveal01.shtml',
 '/players/a/achteaj01.shtml',
 '/players/a/ackerji01.shtml',
 '/players/a/ackledu01.shtml',
 '/players/a/acostma01.shtml',
 '/players/a/acrema01.shtml',
 '/players/a/adamecr01.shtml',
 '/players/a/adamsau01.shtml',
 '/players/a/adamsch01.shtml',
 '/players/a/adamsda02.shtml',
 '/players/a/adamsgl01.shtml',
 '/players/a/adamsla01.shtml',
 '/players/

We define helper functions get_war() and get_rookie_season() for code modularity when scraping. We want to limit our calls to to baseball reference so we define limiting criteria here. We want to make sure we are getting the stats from a players rookie season (>40 games) and their WAR, which is stored in a seperate place on the site.

In [ ]:
def get_War(soup):
  highlights = soup.find("div", attrs={'class' : 'stats_pullout'})
  values = highlights.find('span', attrs={'class' : 'poptip', 'data-tip' : '<strong>Wins Above Replacement</strong><br>A single number that presents the number of wins the player added<br>to the team above what a replacement player (think AAA or AAAA) would add.<br>Scale for a single-season: 8+ MVP Quality, 5+ All-Star Quality, 2+ Starter,<br>0-2 Reserve, < 0 Replacement Level  <br>Developed by Sean Smith of BaseballProjection.com'})

  return float(values.find_next_sibling('p').get_text(strip=True))

def get_rookie_season(df_player):
  df_player = df_player[df_player["Lg"].isin(["AL", 'NL'])] # this filters out the minor league rows
  df_player["G"] = pd.to_numeric(df_player["G"]) # convert games to integer
  df_player = df_player[df_player["G"] >= 40] # filter to seasons over 40 games (rookie qualification)
  return df_player.iloc[0] # get the first 40 game season


Here we scrape the data from Baseball Reference. Player ID's are used in the URL to indicate player page, so we iterate through the file containing player extensions and scrape data for each qualifying player. We want to scrape the feature table and WAR all in one call, so we use the helper functions defined above to successfully scrape the rookie season data and career WAR for each player who had finished their career between 1940 and 2023.

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

trainingData = pd.DataFrame()

with open('playerLinks', 'rb') as pickle_file:
  playerLinks = pickle.load(pickle_file)

for link in playerLinks:
  htmlStr = 'https://www.baseball-reference.com' + link
  try:
    response = requests.get(htmlStr)


    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find_all('table', attrs={'id' : 'batting_standard'})

    df_player = pd.read_html(str(table))[0]
    rookie_season = get_rookie_season(df_player)
    war = get_War(soup)
    rookie_season["WAR"] = war

    trainingData = pd.concat([trainingData, rookie_season.to_frame().T], ignore_index=True) # add this to the dataframe
    time.sleep(3.1)

    print(link)
  except:
    time.sleep(3.1)
    print('pitcher')


trainingData.to_csv('data.csv', index=False)
trainingData

pitcher
pitcher


<ipython-input-3-39553ff2ec0b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_player["G"] = pd.to_numeric(df_player["G"]) # convert games to integer


pitcher
pitcher


<ipython-input-3-39553ff2ec0b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_player["G"] = pd.to_numeric(df_player["G"]) # convert games to integer


/players/a/abbotje01.shtml
pitcher


...

<ipython-input-3-39553ff2ec0b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_player["G"] = pd.to_numeric(df_player["G"]) # convert games to integer


pitcher


<ipython-input-3-39553ff2ec0b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_player["G"] = pd.to_numeric(df_player["G"]) # convert games to integer


/players/a/azocaos01.shtml


,Year,Age,Tm,Lg,G,PA,AB,R,H,2B,...,TB,GDP,HBP,SH,SF,IBB,Pos,Awards,WAR,Unnamed: 30
0,1998,25,CHW,AL,89.0,261.0,244.0,33.0,68.0,14.0,...,120.0,2.0,0.0,2.0,5.0,1.0,897H/D,NaN,-1.4,NaN
1,1994,25,FLA,NL,101.0,371.0,345.0,41.0,86.0,17.0,...,136.0,5.0,5.0,3.0,2.0,1.0,*6/H,NaN,0.5,NaN
2,2006,25,FLA,NL,111.0,281.0,255.0,39.0,54.0,12.0,...,85.0,2.0,3.0,4.0,1.0,2.0,8H9/7,NaN,-0.2,NaN
3,2001,23,TBD,AL,79.0,335.0,304.0,43.0,82.0,17.0,...,116.0,3.0,0.0,3.0,1.0,1.0,4/H,NaN,0.0,NaN
4,1989,23,SDP,NL,57.0,108.0,102.0,13.0,18.0,4.0,...,28.0,1.0,0.0,0.0,1.0,2.0,87H/9,NaN,-1.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1980,28,PHI,NL,51.0,115.0,101.0,12.0,28.0,6.0,...,40.0,3.0,0.0,2.0,2.0,2.0,64H,NaN,-0.1,NaN
96,1979,28,BAL,AL,42.0,94.0,86.0,15.0,22.0,5.0,...,45.0,2.0,0.0,0.0,2.0,1.0,7HD/9,NaN,1.0,NaN
97,2007,23,LAA,AL,79.0,211.0,194.0,18.0,46.0,5.0,...,56.0,8.0,2.0,3.0,2.0,0.0,4H6/7D95,NaN,22.8,NaN
98,2006,23,TOT,NL,79.0,278.0,243.0,32.0,68.0,18.0,...,98.0,7.0,4.0,3.0,0.0,0.0,54H,NaN,2.8,NaN


In [ ]:
df_stats = pd.read_csv('data.csv')
df_stats = df_stats.drop('Unnamed: 30', axis=1)

Here we preview df_stats, which is the df that holds all player information we scrape from Baseball Reference.

In [ ]:
df_stats

,Year,Age,Tm,Lg,G,PA,AB,R,H,2B,...,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos,Awards,WAR
0,1954,20,MLN,NL,122.0,509.0,468.0,58.0,131.0,27.0,...,104.0,209.0,13.0,3.0,6.0,4.0,0.0,*79/H,RoY-4,143.0
1,1962,22,MLN,NL,141.0,382.0,334.0,54.0,77.0,20.0,...,86.0,125.0,10.0,0.0,4.0,3.0,0.0,*37H/45,NaN,-2.8
2,1998,25,CHW,AL,89.0,261.0,244.0,33.0,68.0,14.0,...,104.0,120.0,2.0,0.0,2.0,5.0,1.0,897H/D,NaN,-1.4
3,1994,25,FLA,NL,101.0,371.0,345.0,41.0,86.0,17.0,...,76.0,136.0,5.0,5.0,3.0,2.0,1.0,*6/H,NaN,0.5
4,2006,25,FLA,NL,111.0,281.0,255.0,39.0,54.0,12.0,...,58.0,85.0,2.0,3.0,4.0,1.0,2.0,8H9/7,NaN,-0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4378,1973,24,PIT,NL,103.0,354.0,333.0,44.0,108.0,23.0,...,147.0,175.0,8.0,0.0,0.0,0.0,0.0,97H,RoY-9,25.1
4379,2006,25,TBD,AL,52.0,198.0,183.0,10.0,41.0,6.0,...,48.0,57.0,2.0,0.0,2.0,3.0,1.0,6/H,NaN,44.5
4380,2001,26,CHC,NL,49.0,118.0,106.0,11.0,23.0,3.0,...,83.0,44.0,3.0,3.0,0.0,1.0,1.0,3H,NaN,-0.2
4381,1992,25,BOS,AL,124.0,432.0,392.0,46.0,108.0,19.0,...,85.0,138.0,6.0,4.0,7.0,4.0,1.0,*879H/D,NaN,-0.7
